In [4]:
IMG_SHAPE = [1, 1, 32, 32]
N_RESIDUAL = 4
ADAM_PARAMS = {"lr": 1e-4, "betas": (0.5, 0.9)}

In [1]:
import torch
from torch import nn, optim, autograd

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# device agnostic code
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [55]:
class ResBlock(nn.Module):

    def __init__(self, n_filter=64):
        super(ResBlock, self).__init__()

        self.res_block = nn.Sequential(
            nn.ReLU(True),
            nn.Conv2d(n_filter, n_filter, 5, padding=2),#nn.Linear(DIM, DIM),
            nn.ReLU(True),
            nn.Conv2d(n_filter, n_filter, 5, padding=2),#nn.Linear(DIM, DIM),
        )

    def forward(self, input):
        output = self.res_block(input)
        return input + (0.3*output)

In [56]:
class Discriminator(nn.Module):
    def __init__(self, input_shape, wasserstein=True, n_filter=64):
        super(self.__class__, self).__init__()
        
        self.input_shape = input_shape
        
        self.wasserstein = wasserstein
        self.n_filter = n_filter
        
        self.layers = nn.Sequential(
            nn.Conv2d(input_shape[1], n_filter, 3, padding=1),
            
            nn.Sequential(
                *[
                    nn.Sequential(
                        nn.Conv2d(n_filter, n_filter, 3, padding=1),
                        nn.BatchNorm2d(n_filter),
                        nn.LeakyReLU()
                    )

                    for _ in range(7)
                ]
            )
        )
        
        _, chan, x, y = input_shape
        
        self.linear = nn.Sequential(
            nn.Linear(x * y * n_filter, 1024),
            nn.Sequential(nn.Linear(1024, 1)) if wasserstein else nn.Sequential(nn.Linear(1024, 1), nn.Sigmoid())
        )
                
    def forward(self, input):
        _, _, x, y = self.input_shape
        
        hid = self.layers(input).view(-1, x * y * self.n_filter)
        out = self.linear(hid)
        
        return out

In [57]:
discA = Discriminator(IMG_SHAPE).cuda()

In [2]:
from lib.pic2tensor import pics2array
import matplotlib.pyplot as plt
%matplotlib inline

import os

In [5]:
x_dir = "datasets/notMNIST_small/A/"
y_dir = "datasets/notMNIST_small/B/"

X = np.array(pics2array(x_dir, os.listdir(x_dir), IMG_SHAPE[2:]), dtype="float32")[:, np.newaxis, :, :] / 255.
Y = np.array(pics2array(y_dir, os.listdir(y_dir), IMG_SHAPE[2:]), dtype="float32")[:, np.newaxis, :, :] / 255.

In [84]:
def gradient_penalty(disc, real_data, generated_data, device=device):
    batch_size = real_data.size(0)
    
    epsilon = torch.rand(batch_size, 1, device=device, requires_grad=True)
    epsilon_exp = epsilon.expand(batch_size, real_data.nelement()/batch_size).contiguous().view(real_data.size())
    
    interpolates = epsilon_exp * real_data + (1 - epsilon_exp) * generated_data
    
    disc_interpolates = disc(interpolates)
    
    gradients = autograd.grad(outputs=disc_interpolates.mean(), 
                              inputs=interpolates, 
                              create_graph=True, retain_graph=True
    )[0]
    
    gradients = gradients.view(gradients.size(0), -1)
     
    return ((gradients.norm(2, dim=1) - 1) ** 2).mean()

In [85]:
def 

tensor(2.5898, device='cuda:0')